# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chonchi,-42.6128,-73.8092,20.56,49,73,0.51,CL,1704302148
1,1,talnakh,69.4865,88.3972,-27.11,84,99,4.03,RU,1704302148
2,2,belaya kalitva,48.1700,40.7800,-1.50,68,81,2.46,RU,1704302043
3,3,blackmans bay,-43.0167,147.3167,14.07,79,50,3.81,AU,1704302149
4,4,lhuentse,27.6679,91.1839,8.07,96,93,0.62,BT,1704302149


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
all_cities = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    scale=0.7,
    color="City"
)

# Display the map
all_cities

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<33) & (city_data_df["Max Temp"]>21)
                                  & (city_data_df["Wind Speed"]<4.45) & (city_data_df["Cloudiness"]==0)]


# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
129,129,pacocha,-17.5833,-71.3333,24.08,65,0,3.60,PE,1704302169
274,274,sur,22.5667,59.5289,23.27,68,0,1.58,OM,1704302202
351,351,puerto vallarta,20.6204,-105.2307,24.39,68,0,0.89,MX,1704302212
367,367,piti village,13.4626,144.6933,27.49,86,0,2.57,GU,1704302215
422,422,taltal,-25.4000,-70.4833,21.49,65,0,3.56,CL,1704302245
432,432,cabo san lucas,22.8909,-109.9124,23.20,61,0,2.24,MX,1704302099
487,487,jeddah,21.5169,39.2192,24.92,47,0,3.60,SA,1704302078
527,527,essaouira,31.5125,-9.7700,24.72,20,0,3.09,MA,1704302263


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.loc[:,["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
129,pacocha,PE,-17.5833,-71.3333,65,
274,sur,OM,22.5667,59.5289,68,
351,puerto vallarta,MX,20.6204,-105.2307,68,
367,piti village,GU,13.4626,144.6933,86,
422,taltal,CL,-25.4000,-70.4833,65,
432,cabo san lucas,MX,22.8909,-109.9124,61,
487,jeddah,SA,21.5169,39.2192,47,
527,essaouira,MA,31.5125,-9.7700,20,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude=row["Lat"]
    longitude=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
        
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
pacocha - nearest hotel: Gran Hotel Ilo
sur - nearest hotel: Sur Hotel
puerto vallarta - nearest hotel: Buenaventura Grand
piti village - nearest hotel: The Cliff Hotel
taltal - nearest hotel: Hostería Taltal
cabo san lucas - nearest hotel: Comfort Rooms
jeddah - nearest hotel: No hotel found
essaouira - nearest hotel: Essaouira Wind Palace


,City,Country,Lat,Lng,Humidity,Hotel Name
129,pacocha,PE,-17.5833,-71.3333,65,Gran Hotel Ilo
274,sur,OM,22.5667,59.5289,68,Sur Hotel
351,puerto vallarta,MX,20.6204,-105.2307,68,Buenaventura Grand
367,piti village,GU,13.4626,144.6933,86,The Cliff Hotel
422,taltal,CL,-25.4000,-70.4833,65,Hostería Taltal
432,cabo san lucas,MX,22.8909,-109.9124,61,Comfort Rooms
487,jeddah,SA,21.5169,39.2192,47,No hotel found
527,essaouira,MA,31.5125,-9.7700,20,Essaouira Wind Palace


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
hotels_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    color="City",
    hover_cols = ["Hotel Name","City"]
)

# Display the map
hotels_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)